# LAB 02: STD INFECTION RATES

## Thông tin người thực hiện
|  MSSV |Họ và tên   |
|---|---|
|  20120095 |Ngô Quang Hưng   |
|  20120519|Nguyễn Thị Thúy Liễu   |
|  20120540|Võ Hoàng Thảo Nguyên   |
|  20120633|Viên Hải Yến   |
|  20120634|Lê Minh Trí   |


## Mô tả đồ án

Import các thư viện

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from textwrap import wrap
from wordcloud import WordCloud
import plotly.express as px
import matplotlib.colors as colors
import warnings

### <span style="color:yellow"> A. Thu thập dữ liệu</span>

#### Ngữ cảnh thực hiện tìm kiếm dữ liệu

#### Chủ đề

Nguồn

#### Người ta có cho phép sử dụng dữ liệu như thế này hay không?

#### Người ta đã thu thập dữ liệu này như thế nào? Phương pháp thực hiện là gì?

#### Đọc dữ liệu

----

### <span style="color:yellow"> B. Khám phá dữ liệu</span>

#### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

#### Mỗi cột có ý nghĩa gì?

#### Thông tin mỗi cột

#### Mỗi cột hiện đang có kiểu dữ liệu gì?

#### Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp hay không?

#### Phân bố dữ liệu ở các cột

##### Với mỗi cột có kiểu dữ liệu dạng numeric, các giá trị được phân bố như thế nào?

##### Với mỗi cột không có kiểu dữ liệu dạng categorical, các giá trị được phân bố như thế nào?

#### Tiền xử lí dữ liệu

### <span style="color:yellow"> C. Khám phá mối quan hệ trong dữ liệu</span>